In [1]:
import os
import numpy as np
import pandas as pd
unbill=pd.read_csv("E:\\D science @ DVS\\Datasets\\Datasets\\unbilled.csv")

In [2]:
# using get dummies method for catagerical varibale alone we can create dummy varibales

unbill1=pd.get_dummies(unbill)

In [3]:
unbill.shape
unbill1.shape

(35375, 34)

In [4]:
unbill.nunique()

Segment                    5
Billing_Class              2
Site_Billing_Status        3
Site_Status                4
Consumption             9982
Value_Unbilled         16853
Billing_Frequency          2
UnbilledDays            1668
RootCause                 10
Voice_Attempt              4
target                     2
dtype: int64

In [5]:
unbill.dtypes

Segment                 object
Billing_Class           object
Site_Billing_Status     object
Site_Status             object
Consumption            float64
Value_Unbilled         float64
Billing_Frequency       object
UnbilledDays           float64
RootCause               object
Voice_Attempt           object
target                   int64
dtype: object

In [6]:
# get dummies work for only catergirical data and more over the dummy col are limited to 5*(source data set cols)

# in our data we have 11 col max and then we have scope to create 55 col( dummy+main)

# though it is catergerical variable like ID (AI000454 ex) its unique and we should not use this for machine learning model

# PII(adhar number, voter id ect) we should not include that into machine learning model

# if we are adding them that time we will have to create 10000 of dummy variable which is strightly restricted. means derived table can only 5 times of actual data set columns

# in our unbill data set its fine for any other catergerical col when it is having PII information then we have deal it properly

# How to solve this:
#----------------------

# 1). Based on binary dependent variable we can caluate the percentage of these catergerical col and group the data set

# in our case RootCause catergerical column and then target is the binary dependent varible

# Nuumerical values completely exculded

# 


In [7]:
abc=pd.crosstab(unbill["RootCause"],unbill["target"])

In [8]:
abc["index"]=abc.index

In [9]:
abc["event"]=abc[1]

In [10]:
abc

target,0,1,index,event
RootCause,,,,
Billing Block,11,0,Billing Block,0
Business Exception,1125,81,Business Exception,81
EMMA,1729,111,EMMA,111
Impl Perm Est,5631,189,Impl Perm Est,189
Implausible,5904,467,Implausible,467
Lock - Billing,177,35,Lock - Billing,35
Lost Site,102,5,Lost Site,5
No Root Cause Match,16900,713,No Root Cause Match,713
Outsort,2094,82,Outsort,82


In [11]:
abc1=abc[["index","event"]]

In [12]:
abc1

target,index,event
RootCause,,
Billing Block,Billing Block,0
Business Exception,Business Exception,81
EMMA,EMMA,111
Impl Perm Est,Impl Perm Est,189
Implausible,Implausible,467
Lock - Billing,Lock - Billing,35
Lost Site,Lost Site,5
No Root Cause Match,No Root Cause Match,713
Outsort,Outsort,82


In [13]:
abc1["precent_event"]=np.round((abc["event"]/np.sum(abc1["event"])*100),2)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [14]:
abc1

target,index,event,precent_event
RootCause,,,
Billing Block,Billing Block,0,0.00
Business Exception,Business Exception,81,4.77
EMMA,EMMA,111,6.54
Impl Perm Est,Impl Perm Est,189,11.13
Implausible,Implausible,467,27.50
Lock - Billing,Lock - Billing,35,2.06
Lost Site,Lost Site,5,0.29
No Root Cause Match,No Root Cause Match,713,41.99
Outsort,Outsort,82,4.83


In [15]:
# Group the precent_event

In [16]:
def group_cal(prencent_event):
    if prencent_event<25:
        return "group1"
    elif prencent_event<50:
        return "group2"
    elif prencent_event<75:
        return "group3"
    else:
        return "group4"

In [17]:
abc1["groups"]=abc1["precent_event"].apply(group_cal)

In [18]:
abc1

target,index,event,precent_event,groups
RootCause,,,,
Billing Block,Billing Block,0,0.00,group1
Business Exception,Business Exception,81,4.77,group1
EMMA,EMMA,111,6.54,group1
Impl Perm Est,Impl Perm Est,189,11.13,group1
Implausible,Implausible,467,27.50,group2
Lock - Billing,Lock - Billing,35,2.06,group1
Lost Site,Lost Site,5,0.29,group1
No Root Cause Match,No Root Cause Match,713,41.99,group2
Outsort,Outsort,82,4.83,group1


In [19]:
# Merge the data set

bill2=pd.merge(unbill,abc1[["index","groups"]],left_on=["RootCause"],right_on=["index"],how="inner")

In [20]:
bill2.head()

,Segment,Billing_Class,Site_Billing_Status,Site_Status,Consumption,Value_Unbilled,Billing_Frequency,UnbilledDays,RootCause,Voice_Attempt,target,index,groups
0,Others,Small and Medium Enterprises,Live - Billed & Stop,Clean,1285.0,134.70,Q,144.0,No Root Cause Match,NaN,0,No Root Cause Match,group2
1,Others,Small and Medium Enterprises,Live - Billed & Stop,Clean,1285.0,134.70,Q,144.0,No Root Cause Match,NaN,0,No Root Cause Match,group2
2,Others,Small and Medium Enterprises,Live - Billed & Stop,Dirty,1348.0,154.07,Q,174.0,No Root Cause Match,NaN,0,No Root Cause Match,group2
3,Others,Small and Medium Enterprises,Live - Billed & Stop,Clean,1285.0,134.70,Q,144.0,No Root Cause Match,NaN,0,No Root Cause Match,group2
4,Others,Small and Medium Enterprises,Live - Billed & Stop,Dirty,0.0,96.83,Q,174.0,No Root Cause Match,NaN,0,No Root Cause Match,group2


In [21]:
bill2.shape

(35375, 13)

In [22]:
bill2.dtypes

Segment                 object
Billing_Class           object
Site_Billing_Status     object
Site_Status             object
Consumption            float64
Value_Unbilled         float64
Billing_Frequency       object
UnbilledDays           float64
RootCause               object
Voice_Attempt           object
target                   int64
index                   object
groups                  object
dtype: object

In [23]:
bill3=pd.get_dummies(bill2)

In [24]:
bill3.shape

(35375, 46)

In [25]:
bill3.head()

,Consumption,Value_Unbilled,UnbilledDays,target,Segment_Medium Enterpris,Segment_Medium Enterprise,Segment_Others,Segment_Segment TBA,Segment_Small Enterprise,Billing_Class_Industrial & Commercial,...,index_EMMA,index_Impl Perm Est,index_Implausible,index_Lock - Billing,index_Lost Site,index_No Root Cause Match,index_Outsort,index_Technical Exception,groups_group1,groups_group2
0,1285.0,134.70,144.0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,1
1,1285.0,134.70,144.0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,1
2,1348.0,154.07,174.0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,1
3,1285.0,134.70,144.0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,1
4,0.0,96.83,174.0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,1


In [26]:
# For catergerical varibale with no dependent varaible if we are using in the sesne then such cases we have to go for % wise labeling

In [ ]:
bill3.drop

In [27]:
# The groups column will be used further to create dummy variable (always dummy cols of interger type only)

# What if we dont have binary dependent variable in that case we have to use LABLE (labeling)

bill3["perenctage_col"]=np.round(unbill["RootCause"].value_counts()/len(unbill)*100,2)

In [28]:
# now we can divided the percentages and then group them into different variable

In [29]:
def group_cal(prencent_event):
    if prencent_event<25:
        return "group1"
    elif prencent_event<50:
        return "group2"
    elif prencent_event<75:
        return "group3"
    else:
        return "group4"

In [30]:
# To remove u